In [33]:
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Sets
model.i = RangeSet(1, 3)  
model.j = RangeSet(1, 3) 
model.m = RangeSet(1, 4) 

model.k = RangeSet(1, 2)
model.l = RangeSet(1, 2)
model.n = RangeSet(1, 2)

M = 100

# Parameters
a = {
  (1, 1, 1): 0.05,
  (1, 1, 2): 0.1,
  (1, 1, 3): 0,
  (1, 1, 4): 0.1,
  (1, 2, 1): 0.05,
  (1, 2, 2): 0.1,
  (1, 2, 3): 0.04,
  (1, 2, 4): 0,
  (2, 1, 1): 0.2,
  (2, 1, 2): 0,
  (2, 1, 3): 0.05,
  (2, 1, 4): 0.2,
  (2, 2, 1): 0.2,
  (2, 2, 2): 0.025,
  (2, 2, 3): 0,
  (2, 2, 4): 0.1,
  (2, 3, 1): 0,
  (2, 3, 2): 0.1,
  (2, 3, 3): 0.05,
  (2, 3, 4): 0.05,
  (3, 1, 1): 1,
  (3, 1, 2): 0.5,
  (3, 1, 3): 0.25,
  (3, 1, 4): 0.1,}
b = {
  (1, 1, 1): 2.5,
  (1, 1, 2): 0,
  (1, 2, 1): 0,
  (1, 2, 2): 2,
  (2, 1, 1): 2,
  (2, 1, 2): 1,
}
CmA = {1: 320*5, 2: 240*5, 3: 320*5, 4:400*5}  # Machine capacity Plant A
CnB = {1: 500*5, 2: 400*5}  # Machine capacity Plant B
SmA = {1: 100, 2: 200, 3: 300, 4: 250}  # Machine cost Plant A
SnB = {1: 600, 2: 800}  # Machine cost Plant A
d = {1: 100, 2: 150, 3:200} # cost of materials for semi finished

model.machine_capacity_Plant_A = Param(model.m, initialize=CmA)
model.machine_capacity_Plant_B = Param(model.n, initialize=CnB)
model.machine_cost_Plant_A = Param(model.m, initialize=SmA)
model.machine_cost_Plant_B = Param(model.n, initialize=SnB)
model.machine_time_consumed_for_production_Plant_A = Param(model.i, model.j, model.m, initialize=a) 
model.machine_time_consumed_for_production_Plant_B = Param(model.k, model.l, model.n, initialize=b) 
model.d = Param(model.i, initialize=d)


# Variables
model.x = Var(model.i, model.j, domain=NonNegativeReals)  # Size of production of semi finished products in Plant A.
model.y = Var(model.k, model.l, domain=NonNegativeReals)  # Size of production of semi finished products in Plant B.

# Objective function
def objective_rule(model):
    return sum(model.x[i, j] * model.machine_time_consumed_for_production_Plant_A[i,j,m] * model.machine_cost_Plant_A[m] for m in model.m for i in model.i for j in model.j if (i,j,m) in a) + sum(model.y[k, l] * model.machine_time_consumed_for_production_Plant_B[k,l,n] * model.machine_cost_Plant_B[n] for n in model.n for k in model.k for l in model.l if (k,l,n) in b) + sum(model.x[i,j]*model.d[i] for i in model.i for j in model.j)

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints
# Machine capacity constraints
def machine_capacity_rule_plant_A(model, m):
    return sum(model.x[i, j]*model.machine_time_consumed_for_production_Plant_A[i,j,m] for i in model.i for j in model.j if (i,j,m) in a) <= model.machine_capacity_Plant_A[m]

model.machine_capacity_Plant_A_constraint = Constraint(model.m, rule=machine_capacity_rule_plant_A)

# Supplier capacity constraints
def machine_capacity_rule_plant_B(model, n):
    return sum(model.y[k, l]*model.machine_time_consumed_for_production_Plant_B[k,l,n] for k in model.k for l in model.l if (k,l,n) in b) <= model.machine_capacity_Plant_B[n]

model.machine_capacity_Plant_B_constraint = Constraint(model.n, rule=machine_capacity_rule_plant_B)


model.equation_between_sections_1 = Constraint(expr=sum(model.x[1, j] for j in model.j) == 2*sum(model.y[1,l] for l in model.l) + sum(model.y[2,l] for l in model.l))
model.equation_between_sections_2 = Constraint(expr=sum(model.x[2, j] for j in model.j) == sum(model.y[k,l] for k in model.k for l in model.l))
model.equation_between_sections_3 = Constraint(expr=sum(model.x[3, j] for j in model.j) == sum(model.y[2,l] for l in model.l))
model.equation_between_sections_4 = Constraint(expr=sum(model.y[1,l] for l in model.l) == 200)
model.equation_between_sections_5 = Constraint(expr=sum(model.y[2,l] for l in model.l) == 200)

model.c1 = Constraint(expr=model.x[1,3]==0)
model.c2 = Constraint(expr=model.x[3,2]==0)
model.c3 = Constraint(expr=model.x[3,3]==0)
model.c4 = Constraint(expr=model.y[2,2]==0)



# Solve the model
solver = SolverFactory('cplex')
solver.solve(model)

# Display results
print(value(model.obj))
for i in model.i:
    for j in model.j:
        print(f'Production x[{i},{j}] = {model.x[i,j].value}')

for k in model.k:
    for l in model.l:
        print(f'Procurement y[{k},{l}] = {model.y[k,l].value}')


961200.0
Production x[1,1] = 0.0
Production x[1,2] = 600.0
Production x[1,3] = 0.0
Production x[2,1] = 0.0
Production x[2,2] = 0.0
Production x[2,3] = 400.0
Production x[3,1] = 200.0
Production x[3,2] = 0.0
Production x[3,3] = 0.0
Procurement y[1,1] = 200.0
Procurement y[1,2] = 0.0
Procurement y[2,1] = 200.0
Procurement y[2,2] = 0.0
